Codebook for Keyword Search

Begin with the standard imports.

In [1]:
import numpy as np
import pandas as pd
import openpyxl
from pathlib import Path
import urllib.request

In [1]:
from PyPDF2 import PdfFileMerger 

ModuleNotFoundError: No module named 'PyPDF2'

Take user input for the keyword to be searched. 

In [ ]:
keyword = input('Enter keyword: ')

Create a pandas dataframe from the CORDIS dataset.

In [ ]:
projects = pd.read_excel("data/xlsx/project.xlsx")

In [154]:
projects.columns

Index(['id', 'acronym', 'status', 'title', 'startDate', 'endDate', 'totalCost',
       'ecMaxContribution', 'legalBasis', 'topics', 'ecSignatureDate',
       'frameworkProgramme', 'masterCall', 'subCall', 'fundingScheme',
       'nature', 'objective', 'contentUpdateDate', 'rcn', 'grantDoi'],
      dtype='object')

In [ ]:
projects['objective'][0]

In [ ]:
dsc = pd.Series(projects['objective'], dtype="string")
dsc = dsc.str.lower()
# reg = r"\.|\s[the]\.\s"
counts = dsc.str.count(" " + keyword + " ")
counts = counts[counts>0]
# current implementation ignores cases such as .{term} or {term}. Add regex to overcome these cases. 


In [ ]:
sortedCounts = counts.sort_values(ascending=False)
sortedCounts.index

In [ ]:
df = projects.iloc[sortedCounts.index]

In [153]:
projectList = df[['id', 'title']]
projectList.iloc[[3]]
projectList.columns

Index(['id', 'title'], dtype='object')

In [ ]:
orgs = pd.read_excel("data/xlsx/organization.xlsx")

In [ ]:
t = orgs[orgs['projectID'].isin(projectList['id'])]

In [161]:
companyList = t[['organisationID', 'name']]
#companyList.reset_index(drop=True)
companyList = companyList.drop_duplicates()
companyList

,organisationID,name
94,999528450,FUNDACIO INSTITUT DE BIOENGINYERIA DE CATALUNYA
539,999909175,STMICROELECTRONICS SA
540,936927269,SURECORE LTD
541,999875225,INSTITUT POLYTECHNIQUE DE GRENOBLE
542,999992401,COMMISSARIAT A L ENERGIE ATOMIQUE ET AUX ENERG...
...,...,...
170935,998642064,BLUEFORS CRYOGENICS OY
171564,928209394,ALTERNATIVE ENERGY INNOVATIONS SL
172312,921075820,NYXOAH SA
173026,938857763,SMARTPATCH SRLS


In [165]:
topProjects = projectList.iloc[0:5]
topProjects

,id,title
29174,674434,SMS - Safety Micro Sensor
297,664828,Noisy Electromagnetic Fields - A Technological...
24567,688544,Large Scale Silicon Photonics Matrix for Low P...
28525,888382,Product validation of Software Defined Radio C...
5359,101018843,"Fully integrated and CMOS compatible, nanoscal..."


In [167]:
with pd.ExcelWriter("keywordResults.xlsx") as writer:
    projectList.to_excel(writer, sheet_name="projects", index=False, float_format="%f")
    companyList.to_excel(writer, sheet_name="organizations", index=False, float_format="%f")
    topProjects.to_excel(writer, sheet_name="top_projects", index=False, float_format="%f")

In [170]:
pdf_urls = []
for id in topProjects['id']:
    pdf_urls.append("https://cordis.europa.eu/project/id/" + str(id) + "/en?format=pdf")
pdf_urls

['https://cordis.europa.eu/project/id/674434/en?format=pdf',
 'https://cordis.europa.eu/project/id/664828/en?format=pdf',
 'https://cordis.europa.eu/project/id/688544/en?format=pdf',
 'https://cordis.europa.eu/project/id/888382/en?format=pdf',
 'https://cordis.europa.eu/project/id/101018843/en?format=pdf']